In [17]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import pandas as pd
from langfuse.callback import CallbackHandler
import os
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [18]:
MODEL = "gpt-3.5-turbo"
OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"
CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)
SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summaries"],
    template="""Given a collection of Screen summaries, which describe the same screen:
        {summaries}.
        
        Extract the provided information of the screen and formulate a new description. make sure, that no information is lost"""
)
s2w = pd.read_csv("../data/Screen2Words.csv")
s2w = s2w.drop('Unnamed: 0', axis=1)
sample = pd.read_csv("../data/s2w_sample.csv")
sample = sample.drop("Unnamed: 0", axis=1)

In [19]:
#Sequential chain, of summarizing the Screen2Words Summaries into one and creating a GUI
def s2w_summarized(id, temp=0.5):

    summaries = s2w[s2w["screenId"]==id].values

    llm = ChatOpenAI(model_name=MODEL, temperature=temp)
    
    summary_chain = SUMMARY_PROMPT | llm | StrOutputParser()
    creation_chain = CREATION_PROMPT | llm | StrOutputParser()
    
    chain = {"summary": summary_chain} | RunnablePassthrough.assign(creation=creation_chain)
    
    summaries_str = "\n".join([f"Summary {i+1}: {summary}" for i, summary in enumerate(summaries)])
    
    try: 
        response = chain.invoke({"summaries": summaries_str})#, config={"callbacks":[handler]})
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}_S2W.html", 'w') as f:
        f.write(response.get('creation'))
    return response

In [20]:
#pipeline
def creation_pipeline():
    screens = sample["screenId"].head(10).values
    for screen_id in screens:
        s2w_summarized(screen_id)
        print("f{screen_id}: creation done")

In [21]:
creation_pipeline()

{'summary': 'The screen displays a pop-up or page with multiple share options for sharing an episode through social apps on a train.', 'creation': '<!DOCTYPE html>\n<html>\n<head>\n    <title>Mobile Page - Share Episode</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            background-color: #f2f2f2;\n            margin: 0;\n            padding: 0;\n        }\n        \n        .popup {\n            position: fixed;\n            top: 50%;\n            left: 50%;\n            transform: translate(-50%, -50%);\n            background-color: white;\n            border-radius: 10px;\n            padding: 20px;\n            box-shadow: 0 0 10px rgba(0, 0, 0, 0.3);\n            max-width: 300px;\n        }\n        \n        h1 {\n            font-size: 20px;\n            margin-top: 0;\n        }\n        \n        ul {\n            list-style-type: none;\n            padding: 0;\n            margin: 0;\n        }\n        \n        li {\n            

In [23]:
sample["screenId"].head(10).values


array([ 486,  546,  661,  865,  876,  882, 1050, 1264, 1365, 1929],
      dtype=int64)